In [1]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
import os

# Path ke dataset di Google Drive
zip_path = '/content/drive/My Drive/Dataset/dataset.zip'

# Ekstrak ke folder kerja
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')


Mounted at /content/drive


In [9]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import os

# Step 1: Load the Dataset
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        if os.path.isdir(label_folder):
            for filename in os.listdir(label_folder):
                filepath = os.path.join(label_folder, filename)
                img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    images.append(img)
                    labels.append(label)
    return images, labels

# Step 2: Preprocess the Images
def preprocess_images(images, target_size=(128, 128)):
    resized_images = [cv2.resize(img, target_size) for img in images]
    return resized_images

# Step 3: Extract HOG Features
def extract_hog_features(images):
    hog_features = []
    for img in images:
        feature, _ = hog(img, orientations=9, pixels_per_cell=(8, 8),
                         cells_per_block=(2, 2), block_norm='L2-Hys',
                         visualize=True, channel_axis=None)  # Ganti multichannel dengan channel_axis=None
        hog_features.append(feature)
    return np.array(hog_features)


# Step 4: Load, Preprocess, and Extract Features
dataset_folder = '/content/dataset'
images, labels = load_images_from_folder(dataset_folder)
images = preprocess_images(images)
hog_features = extract_hog_features(images)

# Encode labels as integers
unique_labels = list(set(labels))
label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
int_labels = [label_to_int[label] for label in labels]

# Step 5: Split the Data
X_train, X_test, y_train, y_test = train_test_split(hog_features, int_labels, test_size=0.2, random_state=42)

# Step 6: Train the SVM Model
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train, y_train)

# Step 7: Evaluate the Model
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=unique_labels))




Accuracy: 80.72%
Classification Report:
              precision    recall  f1-score   support

       Sehat       0.88      0.81      0.84        52
      Caries       0.71      0.81      0.76        31

    accuracy                           0.81        83
   macro avg       0.79      0.81      0.80        83
weighted avg       0.81      0.81      0.81        83



In [11]:
# Step 8: Test on New Images
def predict_new_image(image_path, svm_model, label_mapping, target_size=(128, 128)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, target_size)
    feature, _ = hog(img_resized, orientations=9, pixels_per_cell=(8, 8),
                     cells_per_block=(2, 2), block_norm='L2-Hys',
                     visualize=True, channel_axis=None)  # Ubah multichannel menjadi channel_axis=None
    feature = feature.reshape(1, -1)
    prediction = svm_model.predict(feature)
    predicted_label = [key for key, value in label_mapping.items() if value == prediction[0]]
    return predicted_label[0]

new_image_path1 = "karies.jpg"
predicted_label1 = predict_new_image(new_image_path1, svm, label_to_int)
print(f"Predicted label: {predicted_label1}")
print("=================================================================================")
new_image_path2 = "sehat.jpg"
predicted_label2 = predict_new_image(new_image_path2, svm, label_to_int)
print(f"Predicted label: {predicted_label2}")

Predicted label: Caries
Predicted label: Sehat
